In [5]:
import pandas as pd
import numpy as np
import re
import time
from textblob import TextBlob
import sqlite3
import GetOldTweets
import datetime

# Data Preprocessing

In [9]:
database = 'database.sqlite'
conn = sqlite3.connect(database)

In [118]:
#Fetching required data tables
player_data = pd.read_sql("SELECT * FROM Player;", conn)
player_stats_data = pd.read_sql("SELECT * FROM Player_Attributes;", conn)
team_data = pd.read_sql("SELECT * FROM Team;", conn)
match_data = pd.read_sql("SELECT * FROM Match;", conn)
countries = pd.read_sql_query("SELECT * from Country", conn)
leagues = pd.read_sql_query("SELECT * from League", conn)
team_data.head()

,id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
0,1,9987,673.0,KRC Genk,GEN
1,2,9993,675.0,Beerschot AC,BAC
2,3,10000,15005.0,SV Zulte-Waregem,ZUL
3,4,9994,2007.0,Sporting Lokeren,LOK
4,5,9984,1750.0,KSV Cercle Brugge,CEB


In [11]:
# Initializing search parameter data
search_params = pd.DataFrame(columns=['match_api_id', 'date', 'start_date', 'end_date', 'home_team_api_id', 'away_team_api_id', 'home_team_name', 'away_team_name'])
search_params['match_api_id'] = match_data['match_api_id']
search_params['date'] = match_data['date']
search_params['home_team_api_id'] = match_data['home_team_api_id']
search_params['away_team_api_id'] = match_data['away_team_api_id']
search_params.head()

,match_api_id,date,start_date,end_date,home_team_api_id,away_team_api_id,home_team_name,away_team_name
0,492473,2008-08-17 00:00:00,NaN,NaN,9987,9993,NaN,NaN
1,492474,2008-08-16 00:00:00,NaN,NaN,10000,9994,NaN,NaN
2,492475,2008-08-16 00:00:00,NaN,NaN,9984,8635,NaN,NaN
3,492476,2008-08-17 00:00:00,NaN,NaN,9991,9998,NaN,NaN
4,492477,2008-08-16 00:00:00,NaN,NaN,7947,9985,NaN,NaN


In [115]:
# Getting start_date and end_date for search_params
for i in range(0, len(search_params)):
    val = search_params.iloc[i, search_params.columns.get_loc('date')]
    
    # getting first token of date
    date = val.split(' ', 1)[0]
    search_params.iloc[i, search_params.columns.get_loc('date')] = date
    
    # parsing date into year, month, day
    year = date.split('-', 2)[0]
    year = year[2:]
    month = date.split('-', 2)[1]
    day = date.split('-', 2)[2]
    
    # converting date to datetime type
    match_date = datetime.datetime.strptime(year + "-" + month + "-" + day, "%y-%m-%d")
    
    # subtracting 10 days from match_date to get start_date
    start_date = match_date + datetime.timedelta(days=-10)
    search_params.iloc[i, search_params.columns.get_loc('start_date')] = str(start_date).split(' ', 1)[0]
    
    # subtracting 1 day from match_date to get end_date
    end_date = match_date + datetime.timedelta(days=-1)
    search_params.iloc[i, search_params.columns.get_loc('end_date')] = str(end_date).split(' ', 1)[0]
    

In [116]:
search_params.head()

,match_api_id,date,start_date,end_date,home_team_api_id,away_team_api_id,home_team_name,away_team_name
0,492473,2008-08-17,2008-08-07,2008-08-16,9987,9993,KRC Genk,Beerschot AC
1,492474,2008-08-16,2008-08-06,2008-08-15,10000,9994,SV Zulte-Waregem,Sporting Lokeren
2,492475,2008-08-16,2008-08-06,2008-08-15,9984,8635,KSV Cercle Brugge,RSC Anderlecht
3,492476,2008-08-17,2008-08-07,2008-08-16,9991,9998,KAA Gent,RAEC Mons
4,492477,2008-08-16,2008-08-06,2008-08-15,7947,9985,FCV Dender EH,Standard de Liège


In [52]:
# Getting home_team_name and away_team name for search_params
for i in range(0, len(search_params)):   
    # Getting team ids
    home_team_api_id = search_params.iloc[i, search_params.columns.get_loc('home_team_api_id')]
    away_team_api_id = search_params.iloc[i, search_params.columns.get_loc('away_team_api_id')]
    
    # Getting index of ids in team_data
    home_team_index = team_data.index[team_data['team_api_id'] == home_team_api_id]
    away_team_index = team_data.index[team_data['team_api_id'] == away_team_api_id]
    
    # Getting team names
    home_team_name = team_data.iloc[home_team_index, team_data.columns.get_loc('team_long_name')].values[0]
    away_team_name = team_data.iloc[away_team_index, team_data.columns.get_loc('team_long_name')].values[0]
    
    search_params.iloc[i, search_params.columns.get_loc('home_team_name')] = home_team_name
    search_params.iloc[i, search_params.columns.get_loc('away_team_name')] = away_team_name

In [53]:
search_params.head()

,match_api_id,date,start_date,end_date,home_team_api_id,away_team_api_id,home_team_name,away_team_name
0,492473,2008-08-17,2008-08-12,2008-08-16,9987,9993,KRC Genk,Beerschot AC
1,492474,2008-08-16,2008-08-11,2008-08-15,10000,9994,SV Zulte-Waregem,Sporting Lokeren
2,492475,2008-08-16,2008-08-11,2008-08-15,9984,8635,KSV Cercle Brugge,RSC Anderlecht
3,492476,2008-08-17,2008-08-12,2008-08-16,9991,9998,KAA Gent,RAEC Mons
4,492477,2008-08-16,2008-08-11,2008-08-15,7947,9985,FCV Dender EH,Standard de Liège


# Twitter Sentiment Analysis

In [6]:
api = TwitterClient()
for i in range(0, 10):
    start_time = time.time()
    try:
        tweets = api.get_tweets(
            query=search_params.iloc[i, search_params.columns.get_loc('home_team_name')],
            since=search_params.iloc[i, search_params.columns.get_loc('start_date')],
            until=search_params.iloc[i, search_params.columns.get_loc('end_date')],
            count=100)
        if (len(tweets) != 0):
            positive, negative, neutral = get_sentiment(tweets)
        search_params.iloc[i, search_params.columns.get_loc('home_team_positive_tweets')] = positive
        search_params.iloc[i, search_params.columns.get_loc('home_team_negative_tweets')] = negative
        search_params.iloc[i, search_params.columns.get_loc('home_team_neutral_tweets')] = neutral
        
        tweets = api.get_tweets(
            query=search_params.iloc[i, search_params.columns.get_loc('away_team_name')],
            since=search_params.iloc[i, search_params.columns.get_loc('start_date')],
            until=search_params.iloc[i, search_params.columns.get_loc('end_date')],
            count=100)
        if (len(tweets) != 0):
            positive, negative, neutral = get_sentiment(tweets)
        search_params.iloc[i, search_params.columns.get_loc('away_team_positive_tweets')] = positive
        search_params.iloc[i, search_params.columns.get_loc('away_team_negative_tweets')] = negative
        search_params.iloc[i, search_params.columns.get_loc('away_team_neutral_tweets')] = neutral
    except:
        print ("ERROR")
    print('Execution time: {0:.5f} s'.format(time.time() - start_time))

ERROR
Execution time: 0.00079 s
ERROR
Execution time: 0.00008 s
ERROR
Execution time: 0.00008 s
ERROR
Execution time: 0.00007 s
ERROR
Execution time: 0.00007 s
ERROR
Execution time: 0.00007 s
ERROR
Execution time: 0.00007 s
ERROR
Execution time: 0.00007 s
ERROR
Execution time: 0.00017 s
ERROR
Execution time: 0.00008 s


In [2]:
class TwitterClient(object):
    def clean_tweet(self, tweet):
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())
 
    def get_tweet_sentiment(self, tweet):
        '''
        Utility function to classify sentiment of passed tweet
        using textblob's sentiment method
        '''
        # create TextBlob object of passed tweet text
        analysis = TextBlob(self.clean_tweet(tweet))
        # set sentiment
        if analysis.sentiment.polarity > 0:
            return 'positive'
        elif analysis.sentiment.polarity == 0:
            return 'neutral'
        else:
            return 'negative'
        
    def get_tweets(self, query, since, until, count):
        '''
        Main function to fetch tweets and parse them.
        '''
        # empty list to store parsed tweets
        tweets = []
        
        tweetCriteria = GetOldTweets.got.manager.TweetCriteria().setQuerySearch(query).setSince(since).setUntil(until).setMaxTweets(count)
        fetched_tweets = GetOldTweets.got.manager.TweetManager.getTweets(tweetCriteria)
        
        # parsing tweets one by one
        for tweet in fetched_tweets:
            # empty dictionary to store required params of a tweet
            parsed_tweet = {}
 
            # saving text of tweet
            parsed_tweet['text'] = tweet.text
            # saving sentiment of tweet
            parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text)
 
            # appending parsed tweet to tweets list
            if tweet.retweets > 0:
                # if tweet has retweets, ensure that it is appended only once
                if parsed_tweet not in tweets:
                    tweets.append(parsed_tweet)
            else:
                tweets.append(parsed_tweet)
        # return parsed tweets
        return tweets

In [3]:
def get_sentiment(tweets):
    # picking positive tweets from tweets
    ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive']
    # percentage of positive tweets
    print("Positive tweets percentage: {} %".format(100*len(ptweets)/len(tweets)))
    # picking negative tweets from tweets
    ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative']
    # percentage of negative tweets
    print("Negative tweets percentage: {} %".format(100*len(ntweets)/len(tweets)))
    # percentage of neutral tweets
    print("Neutral tweets percentage: {} %".format(100*(len(tweets) - len(ntweets) - len(ptweets))/len(tweets)))
 
    # printing first 5 positive tweets
    print("\n\nPositive tweets:")
    for tweet in ptweets[:10]:
        print(tweet['text'])
 
    # printing first 5 negative tweets
    print("\n\nNegative tweets:")
    for tweet in ntweets[:10]:
        print(tweet['text'])